In [1]:
import pandas as pd


df1= pd.read_csv('../Labb/data/new_dataset1.csv')
df1.head()

,id,age,cholesterol,gluc,smoke,alco,active,cardio,BMI_category_normal range,BMI_category_obese (class I),BMI_category_obese (class II),BMI_category_obese (class III),BMI_category_overweight,BMI_category_underweight,BP_category_elevated,BP_category_hypertension stage I,BP_category_hypertension stage II,BP_category_normal,gender_Men,gender_Women
0,0,50.0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
1,1,55.0,3,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1
2,2,52.0,3,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1
3,3,48.0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0
4,4,48.0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1


In [2]:
df2=pd.read_csv('../Labb/data/new_dataset2.csv')
df2.head()


,id,age,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI,gender_Men,gender_Women
0,0,50.0,110,80,1,1,0,0,1,0,22.0,1,0
1,1,55.0,140,90,3,1,0,0,1,1,34.9,0,1
2,2,52.0,130,70,3,1,0,0,0,1,23.5,0,1
3,3,48.0,150,100,1,1,0,0,1,1,28.7,1,0
4,4,48.0,100,60,1,1,0,0,0,0,23.0,0,1


# 2.4 - Välja modell
Välj 3-5 maskininlärningsmodeller, gärna så olika som möjligt. För varje dataset som vi skapade i uppgift 2.3
gör följande:

- train|validation|test split
- skala datasetet med feature standardization och normalization (de görs inte samtidigt, utan i olika omgångar)
- definiera hyperparametrar (param_grids) att testa för varje modell
- använda GridSearchCV() och välja lämplig evalueringsmetric
- gör prediction på valideringsdata
- beräkna och spara evaluation score för ditt valda metric
- checka bästa parametrarna för respektive modell

Vilket dataset väljer du och vilken modell väljer du? Använd den modellen du valt och träna på all data förutom testdatan.


### Choosen models are:
- 1. Logistic Regression
- 2. KNN 
- 3. SVM -Support Vector Machines
- 4. Random Forest
- 5. Gaussian Naive Bayes

## For first data df1:

- train|validation|test split

In [3]:
print('Number of rows and columns of first dataframe:',df1.shape)

Number of rows and columns of first dataframe: (69971, 20)


In [4]:
from predict_model import split_data

# For df1 data train |val|test split data
X_train1, X_val1, X_test1, y_train1, y_val1, y_test1 = split_data(
    df=df1, target_col="cardio",test_size=0.2, random_state=42)


print(
    f"{X_train1.shape = }\n{X_val1.shape = }\n{X_test1.shape = }\n{y_train1.shape = }\n{y_val1.shape = }\n{y_test1.shape = }\n"
)


X_train1.shape = (55976, 19)
X_val1.shape = (11196, 19)
X_test1.shape = (2799, 19)
y_train1.shape = (55976,)
y_val1.shape = (11196,)
y_test1.shape = (2799,)



- skala datasetet med feature standardization och normalization

#### Feature scaling is standard

In [5]:
from predict_model import scale_features
pipelines = scale_features(scale_type='standard')
pipelines

{'Logistic Regression': Pipeline(steps=[('scaler', StandardScaler()), ('LR', LogisticRegression())]),
 'K-Nearest Neighbor': Pipeline(steps=[('scaler', StandardScaler()), ('KNN', KNeighborsClassifier())]),
 'Decision Tree': Pipeline(steps=[('scaling', StandardScaler()),
                 ('DT', DecisionTreeClassifier())]),
 'Random Forest': Pipeline(steps=[('scaling', StandardScaler()),
                 ('RF', RandomForestClassifier())]),
 'Gaussian Naive Bayes': Pipeline(steps=[('scaling', StandardScaler()), ('NB', GaussianNB())])}

- define hyperparameters for choosen models

In [6]:

# Logistic Regression hyperparameters
log_param_grid = [ {'LR__C': [0.01, 0.1, 1, 10, 100], 'LR__penalty': ['l2'], 'LR__solver': ['lbfgs','newton-cg','sag','saga'],'LR__max_iter': [1000, 5000, 10000]}]

# KNN hyperparameters
knn_param_grid = [{'KNN__n_neighbors': [3, 5, 7, 9, 11], 'KNN__weights': ['uniform', 'distance']}]

# Decision Tree hyperparameters
tree_param_grid = [{'DT__max_depth': [5, 10, 20], 'DT__min_samples_split': [2, 5, 10]}]


# Random Forest hyperparameters
forest_param_grid = [{'RF__n_estimators': [10, 50, 100, 200], 'RF__max_depth': [5, 10, 20], 'RF__min_samples_split': [2, 5, 10]}]


# GaussianNB hyperparameters
Gaussian_param_grid = [{'NB__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}]

In [7]:
param_grids = {
    'Logistic Regression': log_param_grid,
    'K-Nearest Neighbor': knn_param_grid,
    #'Support vector machines linear': svm_param_grid,
    'Decision Tree': tree_param_grid,
    'Random Forest': forest_param_grid,
    'Gaussian Naive Bayes': Gaussian_param_grid
}

In [8]:
from predict_model import grid_search

for model_name, pipeline in pipelines.items():
    param_grid = param_grids[model_name]
    print('=============================================\n')
    print(f'{model_name:}\n')
    score_file='/results/accuracy_scores.txt'
    grid_search(pipeline, param_grid, X_train1, y_train1, X_val1, y_val1,score_file,'dataset1')


Logistic Regression

Accuracy Score: 70.41%
Best parameters: {'LR__C': 1, 'LR__max_iter': 1000, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}

K-Nearest Neighbor



### Feature scaling is minmax

In [ ]:

pipelines_min = scale_features(scale_type='minmax')
pipelines_min

{'Logistic Regression': Pipeline(steps=[('scaler', MinMaxScaler()), ('LR', LogisticRegression())]),
 'K-Nearest Neighbor': Pipeline(steps=[('scaler', MinMaxScaler()), ('KNN', KNeighborsClassifier())]),
 'Decision Tree': Pipeline(steps=[('scaling', MinMaxScaler()), ('DT', DecisionTreeClassifier())]),
 'Random Forest': Pipeline(steps=[('scaling', MinMaxScaler()), ('RF', RandomForestClassifier())]),
 'Gaussian Naive Bayes': Pipeline(steps=[('scaling', MinMaxScaler()), ('NB', GaussianNB())])}

In [ ]:
for model_name, pipeline in pipelines_min.items():
    param_grid = param_grids[model_name]
    print('==============================================\n')
    print(f'{model_name:}\n')
    score_file='/results/accuracy_scores.txt'
    grid_search(pipeline, param_grid, X_train1, y_train1, X_val1, y_val1,score_file,'dataset1')


Logistic Regression

Accuracy Score: 70.40%
Best parameters: {'LR__C': 1, 'LR__max_iter': 5000, 'LR__penalty': 'l2', 'LR__solver': 'sag'}

K-Nearest Neighbor

Accuracy Score: 68.19%
Best parameters: {'KNN__n_neighbors': 11, 'KNN__weights': 'uniform'}

Decision Tree

Accuracy Score: 69.43%
Best parameters: {'DT__max_depth': 5, 'DT__min_samples_split': 2}

Random Forest

Accuracy Score: 70.46%
Best parameters: {'RF__max_depth': 10, 'RF__min_samples_split': 10, 'RF__n_estimators': 200}

Gaussian Naive Bayes

Accuracy Score: 68.47%
Best parameters: {'NB__var_smoothing': 1e-09}


## For first data df2:

- train|validation|test split

In [ ]:
print('Number of rows and columns of second dataframe:',df2.shape)

Number of rows and columns of second dataframe: (69971, 13)


In [ ]:
# For df1 data train |val|test split data
X_train2, X_val2, X_test2, y_train2, y_val2, y_test2 = split_data(
    df=df2, target_col="cardio", test_size=0.2, random_state=42
)

print(
    f"{X_train2.shape = }\n{X_val2.shape = }\n{X_test2.shape = }\n{y_train2.shape = }\n{y_val2.shape = }\n{y_test2.shape = }\n"
)


X_train2.shape = (55976, 12)
X_val2.shape = (11196, 12)
X_test2.shape = (2799, 12)
y_train2.shape = (55976,)
y_val2.shape = (11196,)
y_test2.shape = (2799,)



#### Feature scaling is standard

In [ ]:
# standad scaling for dataset 2
pipelines_stand = scale_features(scale_type='standard')
pipelines_stand

{'Logistic Regression': Pipeline(steps=[('scaler', StandardScaler()), ('LR', LogisticRegression())]),
 'K-Nearest Neighbor': Pipeline(steps=[('scaler', StandardScaler()), ('KNN', KNeighborsClassifier())]),
 'Decision Tree': Pipeline(steps=[('scaling', StandardScaler()),
                 ('DT', DecisionTreeClassifier())]),
 'Random Forest': Pipeline(steps=[('scaling', StandardScaler()),
                 ('RF', RandomForestClassifier())]),
 'Gaussian Naive Bayes': Pipeline(steps=[('scaling', StandardScaler()), ('NB', GaussianNB())])}

In [ ]:

for model_name, pipeline in pipelines_stand.items():
    param_grid = param_grids[model_name]
    print(' ===============================================\n')
    print(f'{model_name:}\n')
    score_file='/results/accuracy_scores.txt'
    grid_search(pipeline, param_grid, X_train2, y_train2, X_val2, y_val2,score_file,'dataset2')


Logistic Regression

Accuracy Score: 72.25%
Best parameters: {'LR__C': 100, 'LR__max_iter': 1000, 'LR__penalty': 'l2', 'LR__solver': 'saga'}

K-Nearest Neighbor

Accuracy Score: 65.47%
Best parameters: {'KNN__n_neighbors': 11, 'KNN__weights': 'distance'}

Decision Tree

Accuracy Score: 73.19%
Best parameters: {'DT__max_depth': 5, 'DT__min_samples_split': 2}

Random Forest

Accuracy Score: 73.49%
Best parameters: {'RF__max_depth': 10, 'RF__min_samples_split': 10, 'RF__n_estimators': 200}

Gaussian Naive Bayes

Accuracy Score: 59.05%
Best parameters: {'NB__var_smoothing': 1e-09}


#### Feature scaling is minmax


In [ ]:
# Min max scaling for dataset 2
pipelines_max = scale_features(scale_type='minmax')
pipelines_max

{'Logistic Regression': Pipeline(steps=[('scaler', MinMaxScaler()), ('LR', LogisticRegression())]),
 'K-Nearest Neighbor': Pipeline(steps=[('scaler', MinMaxScaler()), ('KNN', KNeighborsClassifier())]),
 'Decision Tree': Pipeline(steps=[('scaling', MinMaxScaler()), ('DT', DecisionTreeClassifier())]),
 'Random Forest': Pipeline(steps=[('scaling', MinMaxScaler()), ('RF', RandomForestClassifier())]),
 'Gaussian Naive Bayes': Pipeline(steps=[('scaling', MinMaxScaler()), ('NB', GaussianNB())])}

In [ ]:

for model_name, pipeline in pipelines_max.items():
    param_grid = param_grids[model_name]
    print(' ==================================================\n')
    print(f'{model_name:}\n')
    score_file='/results/accuracy_scores.txt'
    grid_search(pipeline, param_grid, X_train2, y_train2, X_val2, y_val2,score_file,'dataset2')


Logistic Regression

